## Lecture 4 — Data (BCRP + Yahoo) + Plots + Stats + VaR

Reproduce the key parts of the lecture notebook using:

- **Peru (BCRP API)**: `PD04637PD`, `PD04639PD`, `PD04704XD`, `PD04701XD`  
  *(FX + commodities exactly as in the notebook)*
- **USA (yfinance)**: `SPY`, `TLT`, `GLD`

**Deliverables**
- Multiple **plots** (including **one with annotations**)  
- A **summary statistics table**  
- **Historical 95% VaR** for a **60/40 portfolio** (SPY/TLT)

1. Build (and display) the **BCRPData API URL** that requests the 4 series used in the notebook.  


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from scipy import stats

# 1. Construir la URL de la API del BCRP
base_url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api"
series = "PD04637PD-PD04639PD-PD04704XD-PD04701XD"
formato = "json"
periodo = "2023-01-01/2023-12-31" # Ejemplo para el año 2023

url_bcrp = f"{base_url}/{series}/{formato}/{periodo}"
print(f"URL BCRP: {url_bcrp}")

URL BCRP: https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PD04637PD-PD04639PD-PD04704XD-PD04701XD/json/2023-01-01/2023-12-31


2. Download those series and build a **tidy** table: `date`, `series`, `value`.  


In [ ]:
# 2. Descargar y procesar a formato Tidy
url_bcrp = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PD04637PD-PD04639PD-PD04704XD-PD04701XD/json/2023-01-01/2023-12-31"
response = requests.get(url_bcrp).json()
data_list = []
for p in response['periods']:
    date = p['name']
    for i, val in enumerate(p['values']):
        # Limpiamos el valor: si es "n.d." ponemos NaN, de lo contrario a float
        clean_val = float(val) if val != "n.d." else np.nan
        data_list.append({'date': date, 'series_id': i, 'value': clean_val})

df_tidy = pd.DataFrame(data_list)
df_tidy

,date,series_id,value
0,03.Ene.23,0,3.813667
1,03.Ene.23,1,3.812000
2,03.Ene.23,2,380.563998
3,03.Ene.23,3,1843.250000
4,04.Ene.23,0,3.823833
...,...,...,...
1035,08.Dic.23,3,2004.670000
1036,25.Dic.23,0,NaN
1037,25.Dic.23,1,NaN
1038,25.Dic.23,2,385.508159


3. Clean to **wide format** with columns: `fx_interbank`, `fx_sbs`, `gold`, `copper` (as in the notebook).  


In [ ]:
import requests
import pandas as pd
import numpy as np
# 3. Transformar a formato Wide con las columnas exactas
# Mapeo de los IDs de posición a los nombres solicitados
map_names = {0: 'fx_interbank', 1: 'fx_sbs', 2: 'gold', 3: 'copper'}
df_tidy['series_name'] = df_tidy['series_id'].map(map_names)

# Eliminamos duplicados antes del pivot para evitar errores
df_tidy = df_tidy.drop_duplicates(subset=['date', 'series_name'])

# Pivotamos usando los nombres nuevos
df_peru = df_tidy.pivot(index='date', columns='series_name', values='value')

# Limpiar el nombre de las columnas (quita el texto 'series_name' de arriba)
df_peru.columns.name = None

# Corregir las fechas (Español -> Inglés)
meses = {'Ene':'Jan','Feb':'Feb','Mar':'Mar','Abr':'Apr','May':'May','Jun':'Jun',
         'Jul':'Jul','Ago':'Aug','Set':'Sep','Oct':'Oct','Nov':'Nov','Dic':'Dec'}

new_idx = df_peru.index.to_series()
for esp, eng in meses.items():
    new_idx = new_idx.str.replace(esp, eng, regex=False)

# Convertir índice a Datetime
df_peru.index = pd.to_datetime(new_idx, format='%d.%b.%y')
df_peru = df_peru.sort_index().ffill()

# Seleccionar y ordenar las columnas finales
df_peru = df_peru[['fx_interbank', 'fx_sbs', 'gold', 'copper']]

print(df_peru.head())

            fx_interbank  fx_sbs        gold   copper
date                                                 
2023-01-02           NaN     NaN  380.427924  1843.25
2023-01-03      3.813667   3.812  380.563998  1843.25
2023-01-04      3.823833   3.822  372.353980  1857.30
2023-01-05      3.824333   3.824  381.924779  1834.00
2023-01-06      3.798000   3.800  379.316619  1852.20


4. Download `SPY`, `TLT`, `GLD` from yfinance and build: `date`, `ticker`, `close`.  


In [ ]:
import warnings
warnings.filterwarnings('ignore') # Oculta los warnings de futuras versiones

# Descarga silenciosa (progress=False)
df_usa_wide = yf.download(['SPY', 'TLT', 'GLD'],
                          start="2023-01-01",
                          end="2024-01-01",
                          progress=False)['Close']

# 4. Descargar y transformar a formato Tidy (date, ticker, close)
df_usa_wide = yf.download(['SPY', 'TLT', 'GLD'], start="2023-01-01", end="2024-01-01")['Close']

# .stack() pasa los tickers de las columnas a una fila por cada fecha
df_usa_tidy = df_usa_wide.stack().reset_index()
df_usa_tidy.columns = ['date', 'ticker', 'close']

print("Formato Tidy solicitado:")
print(df_usa_tidy.head())

[*********************100%***********************]  3 of 3 completed

Formato Tidy solicitado:
        date ticker       close
0 2023-01-03    GLD  171.059998
1 2023-01-03    SPY  366.069092
2 2023-01-03    TLT   90.171211
3 2023-01-04    GLD  172.669998
4 2023-01-04    SPY  368.895233


5. Compute **daily returns** by ticker (`ret`) and validate there are **no inf values**.  


In [ ]:
# Calculamos retornos en el formato wide para facilitar operaciones
df_ret = df_usa_wide.pct_change().dropna()
print(df_ret)
# Validar que no hay infinitos
has_inf = np.isinf(df_ret).values.any()
print(f"\nPunto 5 - ¿Existen valores infinitos en los retornos?: {has_inf}")

Ticker           GLD       SPY       TLT
Date                                    
2023-01-04  0.009412  0.007720  0.013700
2023-01-05 -0.012451 -0.011413  0.004181
2023-01-06  0.018707  0.022932  0.018397
2023-01-09  0.002245 -0.000567  0.005324
2023-01-10  0.003676  0.007013 -0.016550
...              ...       ...       ...
2023-12-22  0.004434  0.002010 -0.004549
2023-12-26  0.007621  0.004222  0.002843
2023-12-27  0.004538  0.001808  0.017720
2023-12-28 -0.005815  0.000378 -0.007263
2023-12-29 -0.001567 -0.002895 -0.009020

[249 rows x 3 columns]

Punto 5 - ¿Existen valores infinitos en los retornos?: False


6. *(Quantities)* Compare FX levels in Peru: produce a **plot** and a short comment.  


7. *(Proportions)* Compute the **share of positive-return days** by ticker (USA).  


8. Plot that share as a **bar chart** and add **labels above each bar** (`annotate`).  


9. *(Distributions)* Compare the distribution of **Peru Gold** vs **GLD** (histogram).  


10. Add an **ECDF** (if used in the notebook) and comment on what changes vs the histogram.  


11. *(Relationships)* Build `FX_change` and relate it to `SPY_ret` (scatter plot).  


12. Compute the **correlation** between `FX_change` and `SPY_ret` and explain the sign.  


13. Estimate a simple regression `FX_change ~ SPY_ret` and interpret the coefficient.  


14. *(Pandas)* Do a selection exercise: `.iloc` (position-based) vs conditional filtering.  


15. Create missing data on purpose in one series and apply imputation (as in the notebook).  


16. Standardize a variable (z-score) and plot **before vs after**.  


17. Find the day with the largest `|SPY_ret|` and **annotate it** in the returns plot (like the exercise).  


18. Save one figure into `/figures` using `savefig` and verify the file exists.  


19. Build a **summary stats table** for returns (mean, sd, p5, p95, etc.).  


20. Compute **historical 95% VaR** for a **60/40 portfolio (SPY/TLT)** and explain what it means.
